In [1]:
!pip install spatial

  Preparing metadata (setup.py) ... done
  Created wheel for spatial: filename=spatial-0.2.0-py3-none-any.whl size=8484 sha256=e763cbc32c75bc3b7f9be51941be7a6f62a2d60ad39af9493912ec654a548ac8
  Stored in directory: /root/.cache/pip/wheels/b3/a9/45/5776d9357e179ab80f60dfbf7d002d095f1a7c5d8f86104bd3
Successfully built spatial


In [2]:
!pip install scipy

In [3]:
!pip install sktime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.9/28.9 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.2/136.2 kB 8.4 MB/s eta 0:00:00


In [4]:
!pip install dtaidistance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 2.4 MB/s eta 0:00:00


In [ ]:
# The idea of this notebook and using the pretrained model VGG16 is derived from this article: https://medium.com/@jeff.lee.1990710/image-similarity-using-vgg16-transfer-learning-and-cosine-similarity-98571d8055e3

In [5]:
# Import the necessary packages, modules, functions, and methods
import pandas as pd
import numpy as np
import keras
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import h5py
import torch
import torchvision
import cv2
import spatial
import scipy.spatial.distance
from keras.layers import Flatten, Dense, Input,concatenate
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout
from keras.models import Model
from keras.models import Sequential
import tensorflow as tf
import sktime
from sktime.distances import euclidean_distance
from dtaidistance import dtw
import dtaidistance
from sklearn import preprocessing

In [6]:
# mount my google drive so I can import files from there
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# we load the VGG16 pre-trained model
vgg16 = keras.applications.VGG16(weights='imagenet', include_top=True, pooling='max', input_shape=(224, 224, 3))

553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [9]:
#show the layers of VGG16
vgg16.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ fc1 (Dense)                          │ (None, 4096)                │     102,764,544 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 138,357,544 (527.79 MB)

 Trainable params: 138,357,544 (527.79 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# not all layers of the model are used. We extract the vector from layer “fc2” as an extracted feature of an image
model = Model(inputs=vgg16.input, outputs=vgg16.get_layer('fc2').output)

In [ ]:
#below we define three functions that we use to calcualte similarity

In [11]:
# function retrieves the feature vector of an image
def get_feature_vector(img):
  # Convert PyTorch tensor to NumPy array
  img_np = img.permute(1, 2, 0).numpy() # Change the order of dimensions
  img1 = cv2.resize(img_np, (224, 224))
  # Convert NumPy array back to PyTorch tensor
  img1_tensor = torch.from_numpy(img1).permute(2, 0, 1) # Change the order of dimensions
  feature_vector = model.predict(img1_tensor.reshape(1, 224, 224, 3))
  return feature_vector.flatten() #copy of the array collapsed into one dimension

In [12]:
# function for calculating cosine similarity of vectors
def cosine_similarity(vector1, vector2):
  return 1 - scipy.spatial.distance.cosine(vector1, vector2) # Use scipy.spatial.distance to caluate the cosine similarity of the vectors

In [14]:
# function for calculating euclidean similarity of vectors
def euclidean_similarity(x, y):
    return 1 / (1 + euclidean_distance(x, y)) # Use sktime.distances to caluate the euclidean similarity of the vectors

In [15]:
# function for calculating dynamic time warping similarity of vectors
def dtw_similarity(time_series_a,time_series_b):
  # dtw.warping_paths returns the DTW distance and the DTW cost matrix
  distance, paths = dtw.warping_paths(time_series_a.flatten(), time_series_b.flatten(), use_c=False)
  # Compute the optimal path from the DTW cost matrix
  best_path = dtw.best_path(paths)
  return 1 - (distance / len(best_path)) # Use dtaidistance to caluate the dynamic time warping similarity of the vectors

# function for scaling data in an array
def normalize_series(data):
  # collapse the array into one dimension
  data_array = data.flatten()
  # Normalize the data using MinMaxScaler
  scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
  # Fit the scaler on the data and transform it
  normalized_array = scaler.fit_transform(data_array.reshape(-1, 1))
  return normalized_array # return the normalized array

In [ ]:
#We check the similarity between similar songs involved in famous court cases using cosine similarity, euclidean similarity, and dynamic time warping similarity
#In each case the closer the score is to 0 the more dissimilar the songs are while the closer the score is to 1 the more similiar the songs are

In [16]:
# Vanilla Ice's Ice Ice Baby vs. Under Pressure by Queen and David Bowie
copy1 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/IceIceBaby_full.jpg")
org1 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/UnderPressure_full.jpg")
f1 = get_feature_vector(org1)
f2 = get_feature_vector(copy1)
cosine_similarity(f1, f2), euclidean_similarity(f1, f2), dtw_similarity(normalize_series(f1),normalize_series(f2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step


(0.681694283586093, 0.015101140753910863, 0.9993995682880646)

In [17]:
# the Verve's Bitter Sweet Symphony vs. the Rolling Stone's The Last Time
org2 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/BittersweetSymphony_full.jpg")
copy2 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/TheLastTime_full.jpg")
v1 = get_feature_vector(org2)
v2 = get_feature_vector(copy2)
cosine_similarity(v1, v2), euclidean_similarity(v1, v2), dtw_similarity(normalize_series(v1), normalize_series(v2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step


(0.8830819925698183, 0.023915723883511965, 0.9994410148634992)

In [18]:
# Robin Thicke featuring rapper T.I. and singer Pharrell Williams's Blurred Lines vs. Marvin Gaye's Gotta Give It Up
org3 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/GottaGiveItUp_full.jpg")
copy3 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/blurredlines_full.jpg")
a1 = get_feature_vector(org3)
a2 = get_feature_vector(copy3)
cosine_similarity(a1, a2), euclidean_similarity(a1, a2), dtw_similarity(normalize_series(a1), normalize_series(a2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step


(0.8495131125308721, 0.02190357355248741, 0.999454778470172)

In [19]:
# Killing Joke‘s Eighties vs. Nirvana's Come as You are
copy4 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/ComeAsYouAre_full.jpg")
org4 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/TheKillingJoke_full.jpg")
b1 = get_feature_vector(org4)
b2 = get_feature_vector(copy4)
cosine_similarity(b1, b2), euclidean_similarity(b1, b2), dtw_similarity(normalize_series(b1), normalize_series(b2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step


(0.8636194586327166, 0.022366261245836527, 0.999450637698725)

In [20]:
# Chuck Berry's You Can't Catch Me vs. The Beatles Come Together
org5 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/YouCantCatchMe_full.jpg")
copy5 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/ComeTogether_full.jpg")
c1 = get_feature_vector(org5)
c2 = get_feature_vector(copy5)
cosine_similarity(c1, c2), euclidean_similarity(c1, c2), dtw_similarity(normalize_series(c1), normalize_series(c2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step


(0.686388235230803, 0.01563043431759674, 0.9993779478425303)

In [21]:
# Radiohead's Creep vs. Lana Del Rey's Get Free
org6 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Creep_full.jpg")
copy6 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/GetFree_full.jpg")
d1 = get_feature_vector(org6)
d2 = get_feature_vector(copy6)
cosine_similarity(d1, d2), euclidean_similarity(d1, d2), dtw_similarity(normalize_series(d1), normalize_series(d2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step


(0.6375096572500354, 0.017062765589763523, 0.9994060618825654)

In [22]:
# Radiohead's Creep vs. The Hollies The Air That I Breathe
org7 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/TheAirThatIBreathe_full.jpg")
copy7 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Creep_full.jpg")
e1 = get_feature_vector(org7)
e2 = get_feature_vector(copy7)
cosine_similarity(e1, e2), euclidean_similarity(e1, e2), dtw_similarity(normalize_series(e1), normalize_series(e2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step


(0.7093138643210916, 0.01720601420470873, 0.9994540351469776)

In [23]:
# George Harrison's My Sweet Lord vs. the Chiffons’ He’s So Fine
org8 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Hessofine_full.jpg")
copy8 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/MySweetLord_full.jpg")
g1 = get_feature_vector(org8)
g2 = get_feature_vector(copy8)
cosine_similarity(g1, g2), euclidean_similarity(g1, g2), dtw_similarity(normalize_series(g1), normalize_series(g2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step


(0.8456728523696246, 0.02118679329453886, 0.999452541899488)

In [24]:
# Katy Perry featuring Juicy J.'s Dark Horse vs. Flame's Joyful Noise
org9 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/JoyfulNoise_full.jpg")
copy9 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/DarkHorse_full.jpg")
h1 = get_feature_vector(org9)
h2 = get_feature_vector(copy9)
cosine_similarity(h1, h2), euclidean_similarity(h1, h2), dtw_similarity(normalize_series(h1), normalize_series(h2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step


(0.9159587476266267, 0.027243123018305733, 0.9994584938783684)

In [25]:
# Rod Stewart's Do You Think I'm Sexy vs. Jorge Ben jor's Taj Mahal
org10 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/TajMahal_full.jpg")
copy10 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/DoYaThinkImSexy_full.jpg")
k1 = get_feature_vector(org10)
k2 = get_feature_vector(copy10)
cosine_similarity(k1, k2), euclidean_similarity(k1, k2), dtw_similarity(normalize_series(k1), normalize_series(k2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step


(0.8187787326550483, 0.020222149095995016, 0.9994800940960117)

In [26]:
# Ray Parker Jr.'s Ghostbusters vs. Huey Lewis's I Want a New Drug
copy11 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Ghostbusters_full.jpg")
org11 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/IWantaNewDrug_full.jpg")
m1 = get_feature_vector(org11)
m2 = get_feature_vector(copy11)
cosine_similarity(m1, m2), euclidean_similarity(m1, m2), dtw_similarity(normalize_series(m1), normalize_series(m2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step


(0.8285717999075475, 0.02022912358712228, 0.9994001488085716)

In [27]:
# Olivia Rodrigo's Good 4 U vs. Paramore's Misery Business
org12 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/MiseryBusiness_full.jpg")
copy12 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Good4u_full.jpg")
n1 = get_feature_vector(org12)
n2 = get_feature_vector(copy12)
cosine_similarity(n1, n2), euclidean_similarity(n1, n2), dtw_similarity(normalize_series(n1), normalize_series(n2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step


(0.8460560362797562, 0.020749247236828385, 0.9994751331483913)

In [28]:
# Oasis's Shakermaker vs. The New Seekers's I’d Like To Teach The World To Sing
org13 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/I'dLiketoTeaktheworldtosing_full.jpg")
copy13 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Shakermaker_full.jpg")
p1 = get_feature_vector(org13)
p2 = get_feature_vector(copy13)
cosine_similarity(p1, p2), euclidean_similarity(p1, p2), dtw_similarity(normalize_series(p1), normalize_series(p2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step


(0.6232008310320757, 0.014253328163439037, 0.9993905490702446)

In [29]:
# Marvin Gaye's Let's Get It On vs. Ed Sheeran's Thinking Out Loud
org14 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/LetsGetItOn_full.jpg")
copy14 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/ThinkingOutLoud_full.jpg")
y1 = get_feature_vector(org14)
y2 = get_feature_vector(copy14)
cosine_similarity(y1, y2), euclidean_similarity(y1, y2), dtw_similarity(normalize_series(y1), normalize_series(y2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step


(0.8854610382091341, 0.026942234324677832, 0.9994277507884598)

In [30]:
# Led Zeppelin's Stairway to Heaven vs. Spirit's Taurus
org15 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Taurus_full.jpg")
copy15 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/StairwaytoHeaven_full.jpg")
q1 = get_feature_vector(org15)
q2 = get_feature_vector(copy15)
cosine_similarity(q1, q2), euclidean_similarity(q1, q2), dtw_similarity(normalize_series(q1), normalize_series(q2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step


(0.6108721112425285, 0.0165066489673691, 0.9994167724087429)

In [31]:
# Sam Smith's Stay with Me vs. Tom Petty's I won't Back Down
org16 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/IWontBackDown_full.jpg")
copy16 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/StayWithMe_full.jpg")
r1 = get_feature_vector(org16)
r2 = get_feature_vector(copy16)
cosine_similarity(r1, r2), euclidean_similarity(r1, r2), dtw_similarity(normalize_series(r1), normalize_series(r2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step


(0.8793887786519965, 0.02492959299112945, 0.9994885490337766)

In [32]:
# Beach Boy's Surfin USA vs. Chuck Berry's Sweet Little Sixteen
org17 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/SweetLittleSixteen_full.jpg")
copy17 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/SurfinUSA_full.jpg")
s1 = get_feature_vector(org17)
s2 = get_feature_vector(copy17)
cosine_similarity(s1, s2), euclidean_similarity(s1, s2), dtw_similarity(normalize_series(s1), normalize_series(s2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step


(0.9325330746624747, 0.029681654508227722, 0.9995577108138787)

In [33]:
# De La Soul's Transmitting Live From Mars vs. The Turtles's You Showed Me
org18 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/YouShowedMe_full.jpg")
copy18 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/TransmittingLivefromMars_full.jpg")
t1 = get_feature_vector(org18)
t2 = get_feature_vector(copy18)
cosine_similarity(t1, t2), euclidean_similarity(t1, t2), dtw_similarity(normalize_series(t1), normalize_series(t2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step


(0.7083474794205227, 0.019350716849728816, 0.9994543905060796)

In [34]:
# Coldplay's Viva La Vida vs. Joe Satriani's If I Could Fly
org19 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/IfICouldFly_full.jpg")
copy19 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/VivaLaVida_full.jpg")
w1 = get_feature_vector(org19)
w2 = get_feature_vector(copy19)
cosine_similarity(w1, w2), euclidean_similarity(w1, w2), dtw_similarity(normalize_series(w1), normalize_series(w2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step


(0.7167066430692043, 0.015058735503346167, 0.999488394176306)

In [35]:
# Led Zeppelin's Whole Lotta Love vs. Muddy Waters's You Need Love
org20 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/YouNeedLove_full.jpg")
copy20 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/WholeLottaLove_full.jpg")
x1 = get_feature_vector(org20)
x2 = get_feature_vector(copy20)
cosine_similarity(x1, x2), euclidean_similarity(x1, x2), dtw_similarity(normalize_series(x1), normalize_series(x2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step


(0.6981323945472439, 0.016696527662207555, 0.999453701081385)

In [ ]:
# We check the similarity between songs that are very dissimilar to the human ear

In [36]:
# J.S. Bach - Toccata and Fugue in D minor BWV 565 vs. Master Of Puppets - Metallica
org21 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/J.S. Bach - Toccata and Fugue in D minor BWV 565.jpg")
copy21 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Master Of Puppets - Metallica.jpg")
aa1 = get_feature_vector(org21)
aa2 = get_feature_vector(copy21)
cosine_similarity(aa1, aa2), euclidean_similarity(aa1, aa2), dtw_similarity(normalize_series(aa1), normalize_series(aa2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step


(0.7654074180199878, 0.016262855709744582, 0.9994939778198709)

In [37]:
# Michael Jackson - Billie Jean vs. Alice Cooper - Schools Out
org22 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Michael Jackson - Billie Jean.jpg")
copy22 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Alice Cooper - Schools Out.jpg")
bb1 = get_feature_vector(org22)
bb2 = get_feature_vector(copy22)
cosine_similarity(bb1, bb2), euclidean_similarity(bb1, bb2), dtw_similarity(normalize_series(bb1), normalize_series(bb2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step


(0.7912629352891816, 0.01759660541780253, 0.9994249085257664)

In [38]:
# Yakety Sax - Boots Randolph vs. Vanessa Carlton - A Thousand miles
org23 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Yakety Sax - Boots Randolph.jpg")
copy23 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Vanessa Carlton - A Thousand Miles.jpg")
cc1 = get_feature_vector(org23)
cc2 = get_feature_vector(copy23)
cosine_similarity(cc1, cc2), euclidean_similarity(cc1, cc2), dtw_similarity(normalize_series(cc1), normalize_series(cc2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step


(0.8352670693512285, 0.01967795699263267, 0.9994734137275133)

In [39]:
# AC⧸DC - Back In Black vs. 4'33'' - John Cage
org24 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/AC⧸DC - Back In Black.jpg")
copy24 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/4'33'' - John Cage.jpg")
dd1 = get_feature_vector(org24)
dd2 = get_feature_vector(copy24)
cosine_similarity(dd1, dd2), euclidean_similarity(dd1, dd2), dtw_similarity(normalize_series(dd1), normalize_series(dd2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step


(0.3987883689748687, 0.011908708216791283, 0.999200507678238)

In [40]:
# Tequila - The Champs  vs. Chopin - Nocturne in E Flat Major (Op. 9 No. 2)
org25 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Tequila - The Champs.jpg")
copy25 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Chopin - Nocturne in E Flat Major (Op. 9 No. 2).jpg")
ee1 = get_feature_vector(org25)
ee2 = get_feature_vector(copy25)
cosine_similarity(ee1, ee2), euclidean_similarity(ee1, ee2), dtw_similarity(normalize_series(ee1), normalize_series(ee2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step


(0.400440332145656, 0.01183182947837169, 0.9993749291377959)

In [41]:
# Aretha Franklin - Respect vs. Love Theme From Romeo & Juliet - Pyotr Ilyich Tchaikovsky
org26 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Aretha Franklin - Respect.jpg")
copy26 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Love Theme From Romeo & Juliet - Pyotr Ilyich Tchaikovsky.jpg")
ff1 = get_feature_vector(org26)
ff2 = get_feature_vector(copy26)
cosine_similarity(ff1, ff2), euclidean_similarity(ff1, ff2), dtw_similarity(normalize_series(ff1), normalize_series(ff2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step


(0.6903125373272213, 0.018504700600528088, 0.9993435126040547)

In [42]:
# Nirvana - Smells Like Teen Spirit vs. Ave Maria - J.S. Bach and Charles Gounod
org27 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Nirvana - Smells Like Teen Spirit.jpg")
copy27 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Ave Maria - J.S. Bach and Charles Gounod.jpg")
gg1 = get_feature_vector(org27)
gg2 = get_feature_vector(copy27)
cosine_similarity(gg1, gg2), euclidean_similarity(gg1, gg2), dtw_similarity(normalize_series(gg1), normalize_series(gg2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step


(0.5266137760286788, 0.0130172093373965, 0.9993828220222525)

In [43]:
# David Bowie - Heroes vs. OutKast - Hey Ya
org28 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/David Bowie - Heroes.jpg")
copy28 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/OutKast - Hey Ya.jpg")
hh1 = get_feature_vector(org28)
hh2 = get_feature_vector(copy28)
cosine_similarity(hh1, hh2), euclidean_similarity(hh1, hh2), dtw_similarity(normalize_series(hh1), normalize_series(hh2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step


(0.7535707971041999, 0.016569117191632586, 0.9994351949051616)

In [44]:
# The Dave Brubeck Quartet - Take Five vs. Public Enemy - Fight the Power
org29 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/The Dave Brubeck Quartet - Take Five.jpg")
copy29 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Public Enemy - Fight the Power.jpg")
jj1 = get_feature_vector(org29)
jj2 = get_feature_vector(copy29)
cosine_similarity(jj1, jj2), euclidean_similarity(jj1, jj2), dtw_similarity(normalize_series(jj1), normalize_series(jj2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step


(0.8214257206410498, 0.02050930579926731, 0.9993971975254913)

In [ ]:
# Grieg - Peer Gynt Suite No. 1, Op. 46 I. Morning Mood vs. Pour Some Sugar On Me - Def Leppard
org30 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Grieg - Peer Gynt Suite No. 1, Op. 46 I. Morning Mood.jpg")
copy30 = torchvision.io.read_image("/content/drive/MyDrive/may-summer-2024-team-good-composers-borrow-great-ones-steal/Curl_Ideas_and_Mentioned_Resources/Mel_Spec_jpgs_of_song_pairs/Pour Some Sugar On Me - Def Leppard.jpg")
kk1 = get_feature_vector(org30)
kk2 = get_feature_vector(copy30)
cosine_similarity(kk1, kk2), euclidean_similarity(kk1, kk2), dtw_similarity(normalize_series(kk1), normalize_series(kk2))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step


(0.4487928296802366, 0.012784468918728666, 0.9993505682925943)

In [ ]:
#The scores are all extremely high (close to 1) for all pairs of songs with respect to DTW similarity.
#The scores are all extremely low (close to 0) for all pairs of songs with respect to euclidean similarity.
#With respect to cosine similarity, for all the similar songs, the similarity is rather high, and for the dissimilar songs, the similarity is overall high except for two pairs of songs.
#This model as-is is not the best representation of similarity between songs but it's not the worst either.